<a href="https://colab.research.google.com/github/Abhi-213/Mini-Project/blob/main/mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle


In [2]:
!mkdir -p ~/.kaggle


In [3]:
!mv kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d vigneshvenkateswaran/bot-iot

Dataset URL: https://www.kaggle.com/datasets/vigneshvenkateswaran/bot-iot
License(s): unknown
100% 1.17G/1.17G [00:54<00:00, 18.4MB/s]
100% 1.17G/1.17G [00:54<00:00, 22.9MB/s]


In [4]:
!unzip /content/bot-iot.zip

Archive:  /content/bot-iot.zip
  inflating: data_1.csv              
  inflating: data_10.csv             
  inflating: data_11.csv             
  inflating: data_12.csv             
  inflating: data_13.csv             
  inflating: data_14.csv             
  inflating: data_15.csv             
  inflating: data_16.csv             
  inflating: data_17.csv             
  inflating: data_18.csv             
  inflating: data_19.csv             
  inflating: data_2.csv              
  inflating: data_20.csv             
  inflating: data_21.csv             
  inflating: data_22.csv             
  inflating: data_23.csv             
  inflating: data_24.csv             
  inflating: data_25.csv             
  inflating: data_26.csv             
  inflating: data_27.csv             
  inflating: data_28.csv             
  inflating: data_29.csv             
  inflating: data_3.csv              
  inflating: data_30.csv             
  inflating: data_31.csv             
  inflating: data_3

In [20]:
import pandas as pd

# Load the dataset
# Replace 'your_dataset.csv' with the actual path to your dataset file
df = pd.read_csv('/content/data_1.csv')

# Drop the specified columns
columns_to_drop = ['smac', 'dmac', 'soui', 'doui', 'sco', 'dco']
df.drop(columns=columns_to_drop, inplace=True)

# Optionally, save the modified dataset to a new CSV file
df.to_csv('modified_dataset.csv', index=False)

# Display the first few rows of the modified dataset
print(df.head())


<ipython-input-20-760be56b5d39>:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data_1.csv')


   pkSeqID         stime flgs proto            saddr  sport           daddr  \
0        1  1.526344e+09    e   arp    192.168.100.1    NaN   192.168.100.3   
1        2  1.526344e+09    e   tcp    192.168.100.7    139   192.168.100.4   
2        3  1.526344e+09    e   udp  192.168.100.149  51838  27.124.125.250   
3        4  1.526344e+09    e   arp    192.168.100.4    NaN   192.168.100.7   
4        5  1.526344e+09    e   udp   192.168.100.27  58999   192.168.100.1   

   dport  pkts  bytes  ... spkts  dpkts  sbytes  dbytes       rate     srate  \
0    NaN     4    240  ...     2      2     120     120   0.002508  0.000836   
1  36390    10    680  ...     5      5     350     330   0.006190  0.002751   
2    123     2    180  ...     1      1      90      90  20.590960  0.000000   
3    NaN    10    510  ...     5      5     210     300   0.006189  0.002751   
4     53     4    630  ...     2      2     174     456   0.005264  0.001755   

      drate  attack  category  subcategory  

In [27]:
# Check the data types of the columns
print(df.dtypes)


pkSeqID           int64
stime           float64
flgs             object
proto             int64
saddr            object
sport            object
daddr            object
dport            object
pkts              int64
bytes             int64
state            object
ltime           float64
seq               int64
dur             float64
mean            float64
stddev          float64
smac            float64
dmac            float64
sum             float64
min             float64
max             float64
soui            float64
doui            float64
sco             float64
dco             float64
spkts             int64
dpkts             int64
sbytes            int64
dbytes            int64
rate            float64
srate           float64
drate           float64
attack            int64
category          int64
subcategory      object
subcategory       int64
dtype: object


In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Custom Callback to display progress bar for accuracy and batch
class AccuracyProgressBar(Callback):
    def __init__(self):
        super().__init__()
        self.epochs = 0
        self.progress_bar = None

    def on_epoch_begin(self, epoch, logs=None):
        self.epochs += 1
        self.progress_bar = tqdm(total=self.params['steps'], position=0, leave=True)
        self.progress_bar.set_description(f'Epoch {self.epochs}/{self.params["epochs"]}')

    def on_batch_end(self, batch, logs=None):
        accuracy = logs.get('accuracy')
        self.progress_bar.set_postfix(accuracy=accuracy)
        self.progress_bar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        self.progress_bar.close()

# Load the dataset
df = pd.read_csv('/content/data_1.csv', low_memory=False)

# Inspect initial DataFrame
print("Initial DataFrame shape:", df.shape)
print("Initial columns:", df.columns)

# Remove specified columns
columns_to_drop = ['smac', 'dmac', 'soui', 'doui', 'sco', 'dco', 'sport', 'dport']
df.drop(columns=columns_to_drop, inplace=True)

# Check for and encode categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
for col in categorical_columns:
    df[col] = LabelEncoder().fit_transform(df[col])

# Convert all other non-numeric columns to numeric, coercing errors
df = df.apply(pd.to_numeric, errors='coerce')

# Check for NaN values after conversion and fill them
print("Checking for NaN values in DataFrame after conversion:")
print(df.isna().sum())
df.fillna(0, inplace=True)  # Fill NaN values with 0

# Split the Data into Features and Labels
columns_to_drop = ['pkSeqID', 'attack', 'category']  # Adjust based on available columns
X = df.drop(columns=columns_to_drop, axis=1, errors='ignore')  # Features
y = df['attack'].values  # Labels

# Split into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for CNN
X_train_cnn = X_train.values.reshape(-1, X_train.shape[1]).astype(np.float32)
X_test_cnn = X_test.values.reshape(-1, X_test.shape[1]).astype(np.float32)

# Reshape for LSTM and RNN
X_train_lstm_rnn = X_train.values.reshape(-1, 1, X_train.shape[1]).astype(np.float32)
X_test_lstm_rnn = X_test.values.reshape(-1, 1, X_test.shape[1]).astype(np.float32)

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Introduce label noise: Flip a small fraction of labels
noise_factor = 0.1  # 10% noise
n_noisy_labels = int(len(y_train) * noise_factor)
indices = np.random.choice(len(y_train), n_noisy_labels, replace=False)
y_train_noisy = y_train.copy()
y_train_noisy[indices] = np.random.choice(np.unique(y_train), n_noisy_labels)

# One-hot encode noisy labels
y_train_noisy_cat = to_categorical(y_train_noisy)

# CNN Model with Reduced Complexity
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Dense(8, activation='relu', input_shape=input_shape))  # Reduced units
    model.add(Dropout(0.7))  # Increased dropout rate
    model.add(Dense(y_train_noisy_cat.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = create_cnn_model((X_train_cnn.shape[1],))
cnn_model.fit(X_train_cnn, y_train_noisy_cat, epochs=5, batch_size=32, verbose=0, callbacks=[AccuracyProgressBar()])

# LSTM Model with Reduced Complexity
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(16, input_shape=input_shape))  # Reduced units
    model.add(Dropout(0.7))  # Increased dropout rate
    model.add(Dense(y_train_noisy_cat.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

lstm_model = create_lstm_model((X_train_lstm_rnn.shape[1], X_train_lstm_rnn.shape[2]))
lstm_model.fit(X_train_lstm_rnn, y_train_noisy_cat, epochs=5, batch_size=32, verbose=0, callbacks=[AccuracyProgressBar()])

# RNN Model with Reduced Complexity
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(16, input_shape=input_shape))  # Reduced units
    model.add(Dropout(0.7))  # Increased dropout rate
    model.add(Dense(y_train_noisy_cat.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

rnn_model = create_rnn_model((X_train_lstm_rnn.shape[1], X_train_lstm_rnn.shape[2]))
rnn_model.fit(X_train_lstm_rnn, y_train_noisy_cat, epochs=5, batch_size=32, verbose=0, callbacks=[AccuracyProgressBar()])

# Evaluate Models
cnn_accuracy = cnn_model.evaluate(X_train_cnn, y_train_noisy_cat, verbose=0)[1]
lstm_accuracy = lstm_model.evaluate(X_train_lstm_rnn, y_train_noisy_cat, verbose=0)[1]
rnn_accuracy = rnn_model.evaluate(X_train_lstm_rnn, y_train_noisy_cat, verbose=0)[1]

# Print model accuracies
print(f"CNN Model Accuracy: {cnn_accuracy:.4f}")
print(f"LSTM Model Accuracy: {lstm_accuracy:.4f}")
print(f"RNN Model Accuracy: {rnn_accuracy:.4f}")

# Majority Voting Classifier
class MajorityVotingClassifier:
    def __init__(self, models):
        self.models = models

    def predict(self, X_cnn, X_lstm_rnn):
        # Get predictions for CNN model
        cnn_pred = np.argmax(self.models[0].predict(X_cnn), axis=1)  # Predictions from CNN
        # Get predictions for LSTM model
        lstm_pred = np.argmax(self.models[1].predict(X_lstm_rnn), axis=1)  # Predictions from LSTM
        # Get predictions for RNN model
        rnn_pred = np.argmax(self.models[2].predict(X_lstm_rnn), axis=1)  # Predictions from RNN

        # Stack predictions and find the majority vote
        predictions = np.vstack([cnn_pred, lstm_pred, rnn_pred])
        majority_votes = np.array([np.bincount(pred).argmax() for pred in predictions.T])  # Majority voting

        return majority_votes

# Create an instance of the voting classifier
voting_classifier = MajorityVotingClassifier(models=[cnn_model, lstm_model, rnn_model])

# Make predictions using the Majority Voting Classifier
final_predictions = voting_classifier.predict(X_test_cnn, X_test_lstm_rnn)

# Print final predictions
print("Final predictions from Majority Voting Classifier:", final_predictions)

# Evaluate the accuracy of the Majority Voting Classifier
accuracy = accuracy_score(y_test, final_predictions)
print(f"Majority Voting Classifier Accuracy: {accuracy:.4f}")


Initial DataFrame shape: (1000000, 35)
Initial columns: Index(['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
       'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
       'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
       'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'attack', 'category', 'subcategory '],
      dtype='object')
Checking for NaN values in DataFrame after conversion:
pkSeqID         0
stime           0
flgs            0
proto           0
saddr           0
daddr           0
pkts            0
bytes           0
state           0
ltime           0
seq             0
dur             0
mean            0
stddev          0
sum             0
min             0
max             0
spkts           0
dpkts           0
sbytes          0
dbytes          0
rate            0
srate           0
drate           0
attack          0
category        0
subcategory     0
dtype: int64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Epoch 5/5: 100%|██████████| 25000/25000 [03:01<00:00, 137.84it/s, accuracy=0.948]
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Epoch 5/5: 100%|██████████| 25000/25000 [04:22<00:00, 95.40it/s, accuracy=0.948] 


CNN Model Accuracy: 0.9481
LSTM Model Accuracy: 0.9481
RNN Model Accuracy: 0.9481
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step
Final predictions from Majority Voting Classifier: [1 1 1 ... 1 1 1]
Majority Voting Classifier Accuracy: 0.9980
